In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from green_city.utils import span
from green_city.plotting import plot_balance
from green_city.simulation import Battery

plt.rcParams['figure.figsize'] = [15, 5]

building_nr = 4

In [ ]:
df = pd.read_csv(f"../data/preprocessed/Building_{building_nr}.csv").astype({'datetime': 'datetime64'}).set_index('datetime')
with open("../data/citylearn_challenge_2021/schema.json") as schema_file:
    schema = json.load(schema_file)

building_properties = schema['buildings'][f'Building_{building_nr}']

## Energy balance without the use of a battery

In [ ]:
#buiding 4 does not seem to have any heating installed.
df["dhw_heating_kWh"].unique()

In [ ]:
timespan = span("2009-02-08", "2009-02-12")
adding = ['equipment_electric_power_kWh', 'dhw_heating_kWh', 'cooling_load_kWh']
substracting = ['solar_generation_kW']
plot_balance(timespan, adding, substracting, df)

In [ ]:
df.loc[timespan, substracting].plot.area()
df.loc[timespan, adding].plot.area()

- the equipment power usage does not look like 'natural' consumption, as this is "strip mall retail (Building_4),"

## Using a battery to manage load

In [ ]:
#trying out the battery model from the CityLearn simulation

battery_params = building_properties["electrical_storage"]['attributes']
battery_params["capacity_loss_coef"] =  battery_params.pop("capacity_loss_coefficient")
battery_params["loss_coef"] =  battery_params.pop("loss_coefficient")

battery = Battery(**battery_params)

print("Charging and discharging the battery")
print(battery._soc, "initial")
battery.charge(9.3)
print(battery._soc, "after charging 9.3")
battery.charge(0.3)
print(battery._soc, "after charging 0.3")
battery.charge(-0.3)
print(battery._soc, "after discharging 0.3")